In [3]:
                                                # IMPORTING LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import os
import ccxt
import datetime as dt
import seaborn as sns
import yfinance as yf

                                                    # CREATING THE RESULTS FOLDER FOR THE PROBLEMS SETS 
results_dir='RESULTS_WEEK_2'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)



In [ ]:
                                    # DAY 1 MERTON JUMP MODEL WITH POISSON JUMPS AND MONTE CARLO PRICING FOR OPTIONS
        
                                    
                                        # INITIALIZING THE IMPORTANT PARAMTERS 
S_0=100                                 # INITIAL STOCK PRICE
RISK_FREE_RATE=0.05                     # 5% RISK FREE T_BILL RATE
VOLATILITY_STOCK=0.20                   # VOLATILITY OF STOCK
STRIKES=np.linspace(80,200,20)          # STRIKE VALUES
DURATION=1                              # TIME TO Maturity
Nofpaths=10000                          # NUMBER OF PATHS
Nofsteps=1000                           # Number of time steps
LAMBDA=0.5                              # poisson jump intensity
JUMP_MEAN=-0.0148                        # JUMP MAGNITUDE MEAN
JUMP_VOLATILTITY=0.10                   # JUMP MAGNITUDE VOLAITILTY


'''MODEL TAKES
1) THE JUMP FOLLOWS THE POSSION INDICATOR PROCESS 
2) THE JUMP SIZE IS A RANDOM NORMAL  VARIABLE WITH A GIVEN MEAN AND VOLATILITY
3) THE JUMP SIZE IS NOT CORRELATED WITH THE STOCK PRICE MOVEMENTS
4) THE VOLATILITY AND THE JUMP PARAMTERS AND NON STOCHASTIC 
5) THE METHOD FOR PATH GENERATION WILL BE EULER DISCRETISATION

'''

def Path_Generation(INITIAL_STOCK,RISK_FREE_RATE,VOLATILITY_STOCK,DURATION,LAMBDA,JUMP_MEAN,JUMP_VOLATILTITY,Nofsteps,Nofpaths):
    STOCK_PATHS=np.zeros((Nofpaths,Nofsteps+1))
    z=np.random.randn(Nofpaths,Nofsteps)
    STOCK_PATHS[:,0]=INITIAL_STOCK
    dt=DURATION/Nofsteps
    poisson=np.random.poisson(LAMBDA,size=(Nofpaths,Nofsteps))
    poisson=np.where(poisson>0,1,0)
    jump_mag=np.random.normal(JUMP_MEAN,JUMP_VOLATILTITY,size=(Nofpaths,Nofsteps))
    time=[i*dt for i in range(Nofsteps+1)]
    PARAMTERS={}
    for I in range(Nofsteps):
        z[:,I]=(z[:,I]-np.mean(z[:,I]))/(np.std(z[:,I]))
        STOCK_PATHS[:,I+1]=STOCK_PATHS[:,I] + STOCK_PATHS[:,I]*(RISK_FREE_RATE-LAMBDA*np.mean(jump_mag[:,I]))*dt + VOLATILITY_STOCK*z[:,I]*np.sqrt(dt) + STOCK_PATHS[:,I]*(np.exp(jump_mag[:,I])-1)*poisson[:,I]
        STOCK_PATHS[:,I+1]=np.maximum(STOCK_PATHS[:,I+1],0)
        
    PARAMTERS['PATHS']=STOCK_PATHS
    PARAMTERS['TIME']=time
    PARAMTERS['POISSON_MATRIX']=poisson
    PARAMTERS['JUMP_MAGNITUDE_MATRIX']=jump_mag
    PARAMTERS['STOCK_MATURITY']=STOCK_PATHS[:,-1]
    return PARAMTERS


def monte_carlo_pricing(type,INITIAL_STOCK,STRIKE,RISK_FREE_RATE,VOLATILITY_STOCK,DURATION,LAMBDA,JUMP_MEAN,JUMP_VOLATILTITY,Nofsteps,Nofpaths):
     parameters=Path_Generation(INITIAL_STOCK,RISK_FREE_RATE,VOLATILITY_STOCK,DURATION,LAMBDA,JUMP_MEAN,JUMP_VOLATILTITY,Nofsteps,Nofpaths)
     S_T=parameters['STOCK_MATURITY']
     type=type.lower()
     if type=='call':
         price=np.exp(-RISK_FREE_RATE*DURATION)*np.mean(np.maximum(S_T-STRIKE,0))
     elif type=='put':
        price=np.exp(-RISK_FREE_RATE*DURATION)*np.mean(np.maximum(STRIKE-S_T,0))
     return price
    
Path_Generation(S_0,RISK_FREE_RATE,VOLATILITY_STOCK,DURATION,LAMBDA,JUMP_MEAN,JUMP_VOLATILTITY,Nofsteps,Nofpaths)['STOCK_MATURITY'].max()


(np.float64(0.9902292263173579), np.float64(0.09928228222780743))